In [1]:
#import relevant libraries
import cv2
import numpy as np
from matplotlib import cm

In [2]:
def convert_to_temperature(pixel_avg):
    """
    Converts pixel value (mean) to temperature (fahrenheit) depending upon the camera hardware
    """
    norm = pixel_avg/255 #normalizing mean_value of pixel
    temp = np.linspace(38,105,10000) #creating a temperature range between min and max temperature range observed from camera
    cmap = cm.get_cmap('gray',10000) #creating colormap to map color to temperature
    x = cmap(range(10000)) #make array of colormap of size 10000
    x = x[:,0] #subset first column of x which contains the color
    difference_array = np.absolute(x-norm) #calculate the difference array 
    index = difference_array.argmin()#find the index of nearest element to the norm in the x
    return  temp[index] #return temperature corresponding to the color

In [3]:
def process_frame(frame):
    """Function take the frame and calculate the tempearture of each person and return the tempearture"""

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    heatmap_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    heatmap = cv2.applyColorMap(heatmap_gray, cv2.COLORMAP_HOT)

    # Binary threshold
    _, binary_thresh = cv2.threshold(heatmap_gray,200, 255, cv2.THRESH_BINARY)

    # Image opening: Erosion followed by dilation
    kernel = np.ones((3, 3), np.uint8)
    image_erosion = cv2.erode(binary_thresh, kernel, iterations=1)
    image_opening = cv2.dilate(image_erosion, kernel, iterations=1)
    
    #Calculate the temperature
    temperature = round(convert_to_temperature(cv2.mean(heatmap_gray, mask=image_opening)[0]),2)
    return temperature 

In [11]:
cap = cv2.VideoCapture(r'C:\Users\this pc\Documents\Imager Data\11.wmv')
whT = 320
confThreshold = 0.3
nmsThreshold = 0.5

classesFile = 'coco - Copy.names'
classNames = []
with open(classesFile,'rt') as f:
    classNames = f.read().rstrip('n').split('n')
#print(classNames)
#print(len(classNames))

modelConfiguration = 'yolov3-tiny.cfg'
modelWeights = 'yolov3-tiny.weights'

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_DEFAULT)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)


def findObjects(outputs,img,temp_list):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]* wT), int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2), int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))


    #print(len(bbox))
    indices = cv2.dnn.NMSBoxes(bbox, confs,confThreshold,nmsThreshold)
    for i in indices:
        box = bbox[i]
        x,y,w,h = box[0], box[1], box[2], box[3]
        
        
        #cv2.putText(img,f'{classNames[classIds[i]].upper()} {int(confs[i]*100)}%',
                    #(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6,(255,0,255),2)
        cropped_image = img[int(y):int(y+h), 
                      int(x):int(x+w)]
        temp = process_frame(cropped_image)
        temp_list.append(temp)
        if temp > 100.4:
            color = (0,0,255)
        else:
            color = (0,255,0)
        cv2.rectangle(img, (x,y),(x+w,y+h),color,2)
        cv2.putText(img, "{} F".format(temp), (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7,color, 2, cv2.LINE_AA)
        

temp_list = []
while True:
    success, img = cap.read()
    
    if not success:
        break

    blob = cv2.dnn.blobFromImage(img, 1/255,(whT,whT),[0,0,0],crop=False)
    net.setInput(blob)

    layerNames = net.getLayerNames()
    outputNames = [layerNames[i - 1] for i in net.getUnconnectedOutLayers()]

    outputs = net.forward(outputNames)
    findObjects(outputs,img,temp_list)

    cv2.imshow('Image', img)
    if cv2.waitKey(50) & 0xFF == ord('q'):
            break  

cap.release()
cv2.destroyAllWindows()